In [11]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [74]:
df = pd.read_csv('../test_scores.csv')

In [75]:
df

,tree_id,category,owner,judge_1,judge_2,judge_3,judge_4,judge_5,judge_6,judge_7
0,101,medium,NaN,3,1,3,2,2,4,4
1,109,conifer,NaN,3,4,3,5,3,5,5
2,110,deciduous,NaN,3,4,3,3,4,3,4
3,111,medium,NaN,3,4,4,4,4,5,4
4,112,conifer,NaN,4,4,5,4,4,5,5
...,...,...,...,...,...,...,...,...,...,...
70,267,deciduous,judge_5,3,2,1,3,4,3,4
71,268,medium,judge_4,4,3,3,4,3,5,5
72,270,deciduous,NaN,4,5,5,4,4,5,5
73,271,conifer,NaN,4,4,5,3,4,3,2


In [76]:
np.unique(df['category'].tolist())

array(['broadleaf', 'conifer', 'deciduous', 'medium', 'shohin'],
      dtype='<U9')

In [77]:
zscored = [stats.mstats.zscore(df['judge_%d' % d]) for d in range(1, 8)]

In [78]:
for d in range(1, 8):
    df['judge_%d_zscored' % d] = zscored[d - 1]

In [80]:
categories = np.unique(df['category'])

In [81]:
avg_zscores = [np.mean([df[['judge_%d_zscored' % d]].iloc[i]  for d in range(1, 8)]) for i in range(len(df))]

df['avg_zscore'] = avg_zscores

In [82]:
overall_ranks = len(df) - np.argsort(avg_zscores).argsort() 

In [83]:
df['overall_rank'] = overall_ranks

In [84]:
df['category_rank'] = np.zeros_like(df['overall_rank'])

for c in categories:
    iscat = df['category'] == c
    dfcat = df[iscat]
    cat_zscores = dfcat['avg_zscore']
    category_ranks = len(dfcat) - np.argsort(cat_zscores).argsort() 
    df.loc[iscat, 'category_rank'] = category_ranks

In [86]:
category_winners = df[df['category_rank'] == 1]

In [88]:
category_winners

,tree_id,category,owner,judge_1,judge_2,judge_3,judge_4,judge_5,judge_6,judge_7,judge_1_zscored,judge_2_zscored,judge_3_zscored,judge_4_zscored,judge_5_zscored,judge_6_zscored,judge_7_zscored,avg_zscore,overall_rank,category_rank
5,113,broadleaf,NaN,3,5,5,4,4,4,4,-0.752785,1.574672,0.888791,0.500868,0.812134,0.332115,-0.094703,0.465870,16,1
24,168,medium,judge_2,4,5,5,4,4,5,5,0.474582,1.574672,0.888791,0.500868,0.812134,1.415098,1.089082,0.965032,6,1
42,202,deciduous,NaN,4,4,4,5,5,5,5,0.474582,0.699854,-0.024350,1.489423,1.880731,1.415098,1.089082,1.003488,5,1
49,217,shohin,NaN,5,5,5,5,4,5,5,1.701948,1.574672,0.888791,1.489423,0.812134,1.415098,1.089082,1.281592,1,1
62,242,conifer,NaN,4,5,5,5,5,5,5,0.474582,1.574672,0.888791,1.489423,1.880731,1.415098,1.089082,1.258911,2,1
